In [74]:
import pandas as pd
import numpy as np
import warnings
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings("ignore")

In [75]:
user = pd.read_csv('../Data/whyout_data/user.csv') # (31177,3)
user_interest = pd.read_csv('../Data/whyout_data/user_interest.csv') # (31178,15)
place = pd.read_csv('../Data/whyout_data/place.csv') # shape(4697,10)
product = pd.read_csv('../Data/whyout_data/product.csv') # shape(5821,11)
video = pd.read_csv('../Data/whyout_data/video.csv') # shape(3250, 9)

In [76]:
user_place = pd.read_csv('../Data/whyout_data/user_place.csv') # (31177,4697)
user_product = pd.read_csv('../Data/whyout_data/user_product.csv') # (31177,5821)
user_video = pd.read_csv('../Data/whyout_data/user_video.csv') # (31177, 3250)

In [77]:
drop_user_place = pd.read_csv('../Data/whyout_data/case2_user_place.csv') # (22420,4697) 
drop_user_product = pd.read_csv('../Data/whyout_data/case2_user_product.csv') # (2994,5821)
drop_user_video = pd.read_csv('../Data/whyout_data/case2_user_video.csv') # (11067, 3250)

In [78]:
drop_user_place_idx = pd.read_csv('../Data/whyout_data/case2_user_place_idx.csv') # (22420,4)
drop_user_product_idx = pd.read_csv('../Data/whyout_data/case2_user_product_idx.csv') # (2294,4)
drop_user_video_idx = pd.read_csv('../Data/whyout_data/case2_user_video_idx.csv') # (11067, 4)

In [79]:
del_data_sgd_place_preds = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_sgd_rating_place.csv')
del_data_sgd_product_preds = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_sgd_rating_product.csv')
del_data_sgd_video_preds = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_sgd_rating_video.csv')

In [80]:
place_user_latent = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_user_latent_place.csv')
product_user_latent = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_user_latent_product.csv')
video_user_latent = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_user_latent_video.csv')

In [81]:
# place 활동내역이 있는 user에게 place 추천
place_user_id = 25031
num_recommendations = 10

def recommend_place(sgd_place_preds, user_id, place_df, ratings_df, idx, num_recommendations):
    if user_id in idx['idx'].values:
        print(f'{user_id}번 유저의 행동이 있습니다.')
        user_place_index = int(idx[idx['idx'] == user_id].iloc[:,2])
        drop_user_place_index = int(idx[idx['idx'] == user_id].iloc[:,3])
        print('user_place_index:', user_place_index)
        #print('drop_user_place_index:', drop_user_place_index)

        # 원본 평점 데이터에서 user_id에 해당하는 행을 DataFrame으로 가져온다.
        user_data = ratings_df.loc[drop_user_place_index]

        # 사용자가 이미 평가한 상품의 인덱스를 추출
        user_history_indices = [int(i) for i in user_data[user_data > 0].index.tolist()]
        user_history_non_indices = [int(i) for i in user_data[user_data <= 0].index.tolist()]
        print(f'이미 평가한 place 길이: {len(user_history_indices)}')
        #print(f'이미 평가한 place:', user_history_indices)
        #print(len(user_history_non_indices),user_history_non_indices)
        non_recommendations = place_df.iloc[user_history_indices]['idx'].tolist()
        recommendations = place_df.iloc[user_history_non_indices]['idx'].tolist()

        print("이미 평가한 place idx:", non_recommendations)
        print("평가 안한 place idx:", recommendations)

        # SGD를 통해 예측된 사용자의 평점을 기반으로 데이터 정렬
        user_predictions = sgd_place_preds.loc[drop_user_place_index]
        user_predictions_filtered = user_predictions.iloc[user_history_non_indices]
        sorted_predictions = user_predictions_filtered.sort_values(ascending=False)
        top_recommendations = sorted_predictions.index.tolist()[:num_recommendations]
        print(top_recommendations)
        recommendations_result = place_df.iloc[top_recommendations]['idx'].tolist()
        print(f"user {user_id}에게 추천해줄 {10}개 place idx : {recommendations_result}")
        return recommendations_result
    else:
        print(f'{user_id}번 유저의 행동이 없습니다.')

predictions = recommend_place(del_data_sgd_place_preds, place_user_id, place, drop_user_place, drop_user_place_idx ,num_recommendations)

25031번 유저의 행동이 있습니다.
user_place_index: 23386
이미 평가한 place 길이: 31
이미 평가한 place idx: [727, 803, 845, 1067, 2632, 2708, 2733, 3063, 3123, 3136, 3331, 3621, 3697, 3713, 3750, 3802, 3861, 4166, 4231, 4258, 4457, 4458, 4509, 4546, 4552, 4553, 4557, 4573, 4599, 4615, 4630]
평가 안한 place idx: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165

In [9]:
# product 활동내역이 있는 user에게 product 추천
product_user_id = 10928
num_recommendations = 10

def recommend_product(sgd_product_preds, user_id, product_df, ratings_df, idx, num_recommendations):
    if user_id in idx['idx'].values:
        print(f'{user_id}번 유저의 행동이 있습니다.')
        user_product_index = int(idx[idx['idx'] == user_id].iloc[:,2])
        drop_user_product_index = int(idx[idx['idx'] == user_id].iloc[:,3])
        print('user_product_index:', user_product_index)
        print('drop_user_product_index:', drop_user_product_index)

        # 원본 평점 데이터에서 user_id에 해당하는 행을 DataFrame으로 가져온다.
        user_data = ratings_df.loc[drop_user_product_index]

        # 사용자가 이미 평가한 상품의 인덱스를 추출
        user_history_indices = [int(i) for i in user_data[user_data > 0].index.tolist()]
        user_history_non_indices = [int(i) for i in user_data[user_data <= 0].index.tolist()]
        print(f'이미 평가한 product 길이: {len(user_history_indices)}')
        print(f'이미 평가한 product:', user_history_indices)
        #print(len(user_history_non_indices),user_history_non_indices)
        non_recommendations = product_df.iloc[user_history_indices]['idx'].tolist()
        recommendations = product_df.iloc[user_history_non_indices]['idx'].tolist()

        print("이미 평가한 product idx:", non_recommendations)
        print("평가 안한 product idx:", recommendations)

        # SGD를 통해 예측된 사용자의 평점을 기반으로 데이터 정렬
        user_predictions = sgd_product_preds.loc[drop_user_product_index]
        user_predictions_filtered = user_predictions.iloc[user_history_non_indices]
        sorted_predictions = user_predictions_filtered.sort_values(ascending=False)
        top_recommendations = sorted_predictions.index.tolist()[:num_recommendations]
        print(top_recommendations)
        recommendations_result = product_df.iloc[top_recommendations]['idx'].tolist()
        print(f"user {user_id}에게 추천해줄 {10}개 product idx : {recommendations_result}")
        return recommendations_result
    else:
        print(f'{user_id}번 유저의 행동이 없습니다.')

predictions = recommend_product(del_data_sgd_product_preds, product_user_id, product, drop_user_product, drop_user_product_idx, num_recommendations)

10928번 유저의 행동이 있습니다.
user_product_index: 9900
drop_user_product_index: 76
이미 평가한 product 길이: 10
이미 평가한 product: [199, 465, 518, 549, 784, 1464, 3004, 3654, 3663, 4280]
이미 평가한 product idx: [359, 629, 682, 713, 948, 1645, 3194, 3955, 3964, 4742]
평가 안한 product idx: [2, 3, 4, 5, 6, 7, 8, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 

In [10]:
# video 활동내역이 있는 user에게 video 추천
video_user_id = 7
num_recommendations = 10

def recommend_video(sgd_video_preds, user_id, video_df, ratings_df, idx, num_recommendations):
    if user_id in idx['idx'].values:
        print(f'{user_id}번 유저의 행동이 있습니다.')
        user_video_index = int(idx[idx['idx'] == user_id].iloc[:,2])
        drop_user_video_index = int(idx[idx['idx'] == user_id].iloc[:,3])
        print('user_video_index:', user_video_index)
        print('drop_user_video_index:', drop_user_video_index)

        # 원본 평점 데이터에서 user_id에 해당하는 행을 DataFrame으로 가져온다.
        user_data = ratings_df.loc[drop_user_video_index]

        # 사용자가 이미 평가한 상품의 인덱스를 추출
        user_history_indices = [int(i) for i in user_data[user_data > 0].index.tolist()]
        user_history_non_indices = [int(i) for i in user_data[user_data <= 0].index.tolist()]
        print(f'이미 평가한 video 길이: {len(user_history_indices)}')
        print('이미 평가한 video:', user_history_indices)
        #print(len(user_history_non_indices),user_history_non_indices)
        non_recommendations = video_df.iloc[user_history_indices]['idx'].tolist()
        recommendations = video_df.iloc[user_history_non_indices]['idx'].tolist()

        print("이미 평가한 video idx:", non_recommendations)
        print("평가 안한 video idx:", recommendations)

        # SGD를 통해 예측된 사용자의 평점을 기반으로 데이터 정렬
        user_predictions = sgd_video_preds.loc[drop_user_video_index]
        user_predictions_filtered = user_predictions.iloc[user_history_non_indices]
        sorted_predictions = user_predictions_filtered.sort_values(ascending=False)
        top_recommendations = sorted_predictions.index.tolist()[:num_recommendations]
        recommendations_result = video_df.iloc[top_recommendations]['idx'].tolist()
        print(f"user {user_id}에게 추천해줄 {10}개 video idx : {recommendations_result}")
        return recommendations_result
    else:
        print(f'{user_id}번 유저의 행동이 없습니다.')

predictions = recommend_video(del_data_sgd_video_preds, video_user_id, video, drop_user_video, drop_user_video_idx, num_recommendations)

7번 유저의 행동이 있습니다.
user_video_index: 5
drop_user_video_index: 2
이미 평가한 video 길이: 23
이미 평가한 video: [79, 80, 94, 101, 121, 152, 198, 225, 238, 261, 331, 336, 341, 391, 413, 773, 839, 960, 1147, 1149, 1256, 1257, 1259]
이미 평가한 video idx: [103, 110, 133, 141, 163, 206, 264, 300, 314, 350, 440, 445, 450, 503, 525, 918, 988, 1116, 1324, 1326, 1437, 1438, 1440]
평가 안한 video idx: [9, 10, 11, 12, 13, 14, 17, 18, 19, 20, 21, 25, 26, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 42, 43, 44, 45, 47, 48, 49, 50, 52, 53, 54, 55, 56, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 102, 111, 112, 113, 114, 122, 123, 126, 127, 128, 129, 130, 131, 132, 134, 135, 137, 138, 139, 140, 142, 143, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 164, 165, 166, 167, 169, 170, 171, 172, 175, 176, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 189, 190, 191, 192, 1

In [11]:
# 모든 데이터 place, product, video 추천
user_id = 7
num_recommendations = 10

def recommend_all(total_sgd_preds, user_id, total_df, ratings_df, idx, num_recommendations):
    if user_id in idx['idx'].values:
        print(f'{user_id}번 유저의 행동이 있습니다.')
        user_video_index = int(idx[idx['idx'] == user_id].iloc[:,2])
        print('user_video_index:', user_video_index)

        # 원본 평점 데이터에서 user_id에 해당하는 행을 DataFrame으로 가져온다.
        user_data = ratings_df.loc[user_video_index]

        # 사용자가 이미 평가한 상품의 인덱스를 추출
        user_history_indices = [int(i) for i in user_data[user_data > 0].index.tolist()]
        user_history_non_indices = [int(i) for i in user_data[user_data <= 0].index.tolist()]
        print(f'이미 평가한 아이템 길이: {len(user_history_indices)}')
        print('이미 평가한 아이템:', user_history_indices)
        #print(len(user_history_non_indices),user_history_non_indices)
        non_recommendations = total_df.iloc[user_history_indices]['idx'].tolist()
        recommendations = total_df.iloc[user_history_non_indices]['idx'].tolist()

        print("이미 평가한 아이템 idx:", non_recommendations)
        print("평가 안한 아이템 idx:", recommendations)

        # SGD를 통해 예측된 사용자의 평점을 기반으로 데이터 정렬
        user_predictions = total_sgd_preds.loc[user_video_index]
        user_predictions_filtered = user_predictions.iloc[user_history_non_indices]
        sorted_predictions = user_predictions_filtered.sort_values(ascending=False)
        top_recommendations = sorted_predictions.index.tolist()[:num_recommendations]
        recommendations_result = total_df.iloc[top_recommendations]['idx'].tolist()
        print(f"user {user_id}에게 추천해줄 {10}개 아이템 idx : {recommendations_result}")
        return recommendations_result
    else:
        print(f'{user_id}번 유저의 행동이 없습니다.')

predictions = recommend_all(full_data_sgd_video_preds, user_id, video, user_video, user, num_recommendations)

NameError: name 'full_data_sgd_video_preds' is not defined

In [ ]:
# 추천 시스템 함수 백업 용도
def recommend_step1(item, sgd_video_preds, user_id, item_df, ratings_df, ratings_df_idx, num_recommendations):
    drop_user_video_index = int(ratings_df_idx[ratings_df_idx['idx'] == user_id].iloc[:,3])
    # 원본 행동 데이터에서 user_id에 해당하는 행을 가져옴
    user_data = ratings_df.loc[drop_user_video_index]
    # 유저가 평가하지 않은 아이템의 index를 가져옴
    user_history_non_indices = [int(i) for i in user_data[user_data <= 0].index.tolist()]
    # user_id에 해당하는 SGD 결과값을 가져온 후, 유저가 평가하지 않은 아이템의 결과값만 뽑아옴
    user_predictions = sgd_video_preds.loc[drop_user_video_index]
    user_predictions_filtered = user_predictions.iloc[user_history_non_indices]
    # SGD 결과값이 높은 순으로 정렬
    sorted_predictions = user_predictions_filtered.sort_values(ascending=False)
    # 상위 N개만큼 뽑아옴
    top_recommendations = sorted_predictions.index.tolist()[:num_recommendations]
    # 아이템 idx 매핑
    recommendations_result = item_df.iloc[top_recommendations]['idx'].tolist()
    #print(f"user {user_id}에게 추천해줄 {10}개 {item} idx : {recommendations_result}")
    return recommendations_result

        
def recommend_step2(dict, item, sgd_video_preds, user_id, new_user_id, item_df, ratings_df, ratings_df_idx, num_recommendations):
    #if new_user_id in ratings_df_idx['idx'].values:
    drop_user_video_index = int(ratings_df_idx[ratings_df_idx['idx'] == new_user_id].iloc[:,3])
    user_data = ratings_df.loc[drop_user_video_index]
    user_history_non_indices = [int(i) for i in user_data[user_data <= 0].index.tolist()]
    user_predictions = sgd_video_preds.loc[drop_user_video_index]
    user_predictions_filtered = user_predictions.iloc[user_history_non_indices]
    sorted_predictions = user_predictions_filtered.sort_values(ascending=False)
    top_recommendations = sorted_predictions.index.tolist()[:num_recommendations]
    recommendations_result = item_df.iloc[top_recommendations]['idx'].tolist()
    return recommendations_result
    # else:
    #     print(f'user {new_user_id}가 {item}에 대한 행동이 없음')
    #     print('else item', item)
    #     item_list = ['place', 'video', 'product']
    #     item_list.remove(item)
    #     # 유사도 계산
    #     new_user_idx = item_user_latent_cos(user_id, item, item_list, dict)
    #     print('new 유저 찾음 :', new_user_idx)
    #     # 새로운 new_user를 찾으면 recommend_step1 실행
    #     recommend_step1(item, dict[item][0], new_user_idx, dict[item][1], dict[item][2], dict[item][3], num_recommendations)
    #     return new_user_idx

def item_user_latent_cos(user_id, original_item, item_list, dict):
    if user_id in dict[item_list[0]][3]['idx'].values:
        item = item_list[0]
    elif user_id in dict[item_list[1]][3]['idx'].values:
        item = item_list[1]
    else:
        """
        모든 아이템에 대한 행동이 없는 유저에게 추천하는 함수 추가 
        """
        print(f'user {user_id}는 모든 아이템에 대한 행동이 없음')

    print(f'2. {item} user latent에서 user {user_id}과 유사한 user 찾기')
    drop_user_place_index = int(dict[item][3][dict[item][3]['idx'] == user_id].iloc[:,3])
    cosine_sim_matrix = cosine_similarity(dict[item][4])
    cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=dict[item][4].index, columns=dict[item][4].index)
    # user_id의 score를 가져옴
    user_similarities = cosine_sim_df.loc[drop_user_place_index]
    # user_id를 선택하지 않도록 -1을 해줌
    user_similarities[drop_user_place_index] = -1
    # 유사도가 높은 순으로 정렬
    sorted_user_similarities = user_similarities.sort_values(ascending=False)
    # 가장 유사한 유저 선택
    for i in range(len(sorted_user_similarities)):
        most_similar_user_id = sorted_user_similarities.index[i]
        new_user_id = int(dict[item][3][dict[item][3].iloc[:,3] == most_similar_user_id].iloc[:,0])
        highest_similarity_score = sorted_user_similarities.iloc[i]
        if new_user_id in dict[original_item][3]['idx'].values:
            # new_user_idx 찾기
            new_user_id = int(dict[item][3][dict[item][3].iloc[:,3] == most_similar_user_id].iloc[:,0])
            print('update new_user_id:', new_user_id)
            break
        else:
            print(f'{new_user_id}가 {original_item}에 대한 행동이 없음')
            
    print(f'user {user_id}과 가장 유사한 user : {new_user_id}, cos : {highest_similarity_score}')
    return new_user_id

def recommendation_system(user_id, item, item_list, dict, num_recommendations):
    if user_id in dict[item][3]['idx'].values:
        recomm_list = recommend_step1(item, dict[item][0], user_id, dict[item][1], dict[item][2], dict[item][3], num_recommendations)
        print(f"user {user_id}에게 추천해줄 {10}개 {item} idx : {recomm_list}")
    else:
        print(f'1. user {user_id}는 {item}에 대한 행동내역이 없음')
        # 아이템 리스트에서 행동이 없는 아이템 제거
        item_list.remove(item)
        new_user_id = item_user_latent_cos(user_id, item, item_list, dict)
        next_item = item_list[0]
        print(f'user {new_user_id}에게 {item} recommend_step2 시작')
        recom_list2 = recommend_step2(dict, item, dict[item][0], user_id, new_user_id, dict[item][1], dict[item][2], dict[item][3], num_recommendations)
        print(f"user {user_id}에게 추천해줄 {num_recommendations}개 {item} idx : {recom_list2}")

In [ ]:
def recommend_step1(item, sgd_video_preds, user_id, item_df, ratings_df, ratings_df_idx, num_recommendations):
    drop_user_video_index = int(ratings_df_idx[ratings_df_idx['idx'] == user_id].iloc[:,3])
    # 원본 행동 데이터에서 user_id에 해당하는 행을 가져옴
    user_data = ratings_df.loc[drop_user_video_index]
    # 유저가 평가하지 않은 아이템의 index를 가져옴
    user_history_non_indices = [int(i) for i in user_data[user_data <= 0].index.tolist()]
    # user_id에 해당하는 SGD 결과값을 가져온 후, 유저가 평가하지 않은 아이템의 결과값만 뽑아옴
    user_predictions = sgd_video_preds.loc[drop_user_video_index]
    user_predictions_filtered = user_predictions.iloc[user_history_non_indices]
    # SGD 결과값이 높은 순으로 정렬
    sorted_predictions = user_predictions_filtered.sort_values(ascending=False)
    # 상위 N개만큼 뽑아옴
    top_recommendations = sorted_predictions.index.tolist()[:num_recommendations]
    # 아이템 idx 매핑
    recommendations_result = item_df.iloc[top_recommendations]['idx'].tolist()
    #print(f"user {user_id}에게 추천해줄 {10}개 {item} idx : {recommendations_result}")
    return recommendations_result

def item_user_latent_cos(user_id, original_item, item_list, dict):
    print('유사도 선택시 item_list:', item_list)
    if user_id in dict[item_list[0]][3]['idx'].values:
        item = item_list[0]
        print(f'user {user_id}는 {item}에 대한 행동이 존재함')
    elif user_id in dict[item_list[1]][3]['idx'].values:
        item = item_list[1]
        print(f'user {user_id}는 {item}에 대한 행동이 존재함')
    else:
        """
        모든 아이템에 대한 행동이 없는 유저에게 추천하는 함수 추가 
        """
        print(f'user {user_id}는 모든 아이템에 대한 행동이 없음')

    print(f'2. {item} user latent에서 user {user_id}과 유사한 user 찾기')
    drop_user_place_index = int(dict[item][3][dict[item][3]['idx'] == user_id].iloc[:,3])
    cosine_sim_matrix = cosine_similarity(dict[item][4])
    cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=dict[item][4].index, columns=dict[item][4].index)
    # user_id의 score를 가져옴
    user_similarities = cosine_sim_df.loc[drop_user_place_index]
    # user_id를 선택하지 않도록 -1을 해줌
    user_similarities[drop_user_place_index] = -1
    # 유사도가 높은 순으로 정렬
    sorted_user_similarities = user_similarities.sort_values(ascending=False)
    # 유사도가 높은 유저를 순서대로 기존의 추천하려는 아이템에 행동이 있는지 확인
    for i in range(len(sorted_user_similarities)):
        most_similar_user_id = sorted_user_similarities.index[i]
        new_user_id = int(dict[item][3][dict[item][3].iloc[:,3] == most_similar_user_id].iloc[:,0])
        highest_similarity_score = sorted_user_similarities.iloc[i]
        
        if new_user_id in dict[original_item][3]['idx'].values:
            # new_user_idx 찾기
            new_user_id = int(dict[item][3][dict[item][3].iloc[:,3] == most_similar_user_id].iloc[:,0])
            print('3. update new_user_id:', new_user_id)
            break
        else:
            print(f'{new_user_id}가 {original_item}에 대한 행동이 없음')
    print(f'4. user {user_id}과 가장 유사한 user : {new_user_id}, cos : {highest_similarity_score}')
    return new_user_id


def recommendation_system(user_id, item, item_list, dict, num_recommendations):
    if user_id in dict[item][3]['idx'].values:
        recomm_list = recommend_step1(item, dict[item][0], user_id, dict[item][1], dict[item][2], dict[item][3], num_recommendations)
        print(f"1. user {user_id}에게 추천해줄 {10}개 {item} idx : {recomm_list}")
    else:
        print(f'1. user {user_id}는 {item}에 대한 행동내역이 없음')
        # 아이템 리스트에서 행동이 없는 아이템 제거
        item_list.remove(item)
        new_user_id = item_user_latent_cos(user_id, item, item_list, dict)
        user_id = new_user_id
        print(f'5. user {new_user_id}에게 {item} recommend_step2 시작')
        recom_list2 = recommend_step1(item, dict[item][0], user_id, dict[item][1], dict[item][2], dict[item][3], num_recommendations)
        print(f"6. user {user_id}에게 추천해줄 {num_recommendations}개 {item} idx : {recom_list2}")

In [ ]:
user_id = 12080
num_recommendations = 10
del_dict = { 'place' : [del_data_sgd_place_preds, place, drop_user_place, drop_user_place_idx, place_user_latent],
         'video' : [del_data_sgd_video_preds, video, drop_user_video, drop_user_video_idx, video_user_latent],
         'product' : [del_data_sgd_product_preds, product, drop_user_product, drop_user_product_idx, product_user_latent]}
item = 'place'
item_list = list(dict.keys())

# user 3은 video만 있음, place, product 없음
# user 20001은 place만 있음, product, video 없음
# user 26232는 product만 있음, place, video 없음 
a = recommendation_system(user_id, item, item_list, dict, num_recommendations)

TypeError: descriptor 'keys' of 'dict' object needs an argument

In [ ]:
def place_user_latent_cos(user_id, data, idx):
    drop_user_place_index = int(idx[idx['idx'] == user_id].iloc[:,3])

    cosine_sim_matrix = cosine_similarity(data)
    cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=data.index, columns=data.index)
    
    # user_id의 score를 가져옴
    user_similarities = cosine_sim_df.loc[drop_user_place_index]
    
    # user_id를 선택하지 않도록 -1을 해줌
    user_similarities[drop_user_place_index] = -1
    
    # user_id와 가장 유사한 유저 선택
    most_similar_user_id = user_similarities.idxmax()
    highest_similarity_score = user_similarities.max()

    # new_user_idx 찾기
    new_user_id = int(idx[idx.iloc[:,3] == most_similar_user_id].iloc[:,0])
    print(f'The user most similar to user {user_id} is user {new_user_id} with a similarity score of {highest_similarity_score}')

    return new_user_id

similar_user_id = place_user_latent_cos(3, place_user_latent, drop_user_place_idx)

TypeError: cannot convert the series to <class 'int'>

In [ ]:
def product_user_latent_cos(user_id, data, idx):
    drop_user_product_index = int(idx[idx['idx'] == user_id].iloc[:,3])

    cosine_sim_matrix = cosine_similarity(data)
    cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=data.index, columns=data.index)
    
    # user_id의 score를 가져옴
    user_similarities = cosine_sim_df.loc[drop_user_product_index]
    
    # user_id를 선택하지 않도록 -1을 해줌
    user_similarities[drop_user_product_index] = -1

    # user_id와 가장 유사한 유저 선택
    most_similar_user_id = user_similarities.idxmax()
    highest_similarity_score = user_similarities.max()

    # new_user_idx 찾기
    new_user_id = int(idx[idx.iloc[:,3] == most_similar_user_id].iloc[:,0])
    print(f'The user most similar to user {user_id} is user {new_user_id} with a similarity score of {highest_similarity_score}')

    return new_user_id

similar_user_id = product_user_latent_cos(26232, product_user_latent, drop_user_product_idx)

NameError: name 'product_user_latent' is not defined

In [ ]:
def video_user_latent_cos(user_id, data, idx):
    drop_user_video_index = int(idx[idx['idx'] == user_id].iloc[:,3])

    cosine_sim_matrix = cosine_similarity(data)
    cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=data.index, columns=data.index)
    
    # user_id의 score를 가져옴
    user_similarities = cosine_sim_df.loc[drop_user_video_index]
    
    # user_id를 선택하지 않도록 -1을 해줌
    user_similarities[drop_user_video_index] = -1
    
    # user_id와 가장 유사한 유저 선택
    most_similar_user_id = user_similarities.idxmax()
    highest_similarity_score = user_similarities.max()

    # new_user_idx 찾기
    new_user_id = int(idx[idx.iloc[:,3] == most_similar_user_id].iloc[:,0])
    print(f'The user most similar to user {new_user_id} is user {similar_user_id} with a similarity score of {highest_similarity_score}')

    return new_user_id

input_user_id = 3
similar_user_id = video_user_latent_cos(input_user_id, video_user_latent, drop_user_video_idx)

In [23]:
import pandas as pd
# user = pd.read_csv('../Data/whyout_data/user.csv') # (31177,3)

# user, place, product, video 메타데이터
user_interest = pd.read_csv('../Data/whyout_data/user_interest.csv') # (31177,4)
place = pd.read_csv('../Data/whyout_data/place.csv') # shape(4697,10)
product = pd.read_csv('../Data/whyout_data/product.csv') # shape(5821,11)
video = pd.read_csv('../Data/whyout_data/video.csv') # shape(3250, 9)

# 유저의 전체 행동데이터 
user_place = pd.read_csv('../Data/whyout_data/user_place.csv') # (31177,4697)
user_product = pd.read_csv('../Data/whyout_data/user_product.csv') # (31177,5821)
user_video = pd.read_csv('../Data/whyout_data/user_video.csv') # (31177, 3250)

# [Case 2] 유저의 행동데이터 (행동 데이터가 없는 유저를 삭제한 데이터)
case2_user_place = pd.read_csv('../Data/whyout_data/case2_user_place.csv') # (22420,4697) 
case2_user_product = pd.read_csv('../Data/whyout_data/case2_user_product.csv') # (2994,5821)
case2_user_video = pd.read_csv('../Data/whyout_data/case2_user_video.csv') # (11067, 3250)

# [Case 2] 유저의 행동데이터 idx (행동 데이터가 없는 유저를 삭제한 데이터의 idx)
case2_user_place_idx = pd.read_csv('../Data/whyout_data/case2_user_place_idx.csv') # (22420,4)
case2_user_product_idx = pd.read_csv('../Data/whyout_data/case2_user_product_idx.csv') # (2294,4)
case2_user_video_idx = pd.read_csv('../Data/whyout_data/case2_user_video_idx.csv') # (11067, 4)

# [Case 2] SGD 결과(R)
case2_sgd_rating_place = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_sgd_rating_place.csv')
case2_sgd_rating_product = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_sgd_rating_product.csv')
case2_sgd_rating_video = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_sgd_rating_video.csv')

# [Case 2] SGD 결과의 user_latent(U)
case2_user_latent_place = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_user_latent_place.csv')
case2_user_latent_product = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_user_latent_product.csv')
case2_user_latent_video = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_user_latent_video.csv')

# [Case 2] SGD 결과의 item_latent(V)
case2_item_latent_place = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_item_latent_place.csv')
case2_item_latent_product = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_item_latent_product.csv')
case2_item_latent_video = pd.read_csv('../Data/whyout_data/sgd_result/del_data/case2_item_latent_video.csv')

KeyboardInterrupt: 

In [97]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import random
import warnings
warnings.filterwarnings("ignore")

def recommend_step(item, user_id, case2_dict):
    """
    case2_dict[item][0] : sgd 결과
    case2_dict[item][1] : meta data
    case2_dict[item][2] : 유저의 행동데이터
    case2_dict[item][3] : 행동데이터의 idx
    case2_dict[item][4] : user latent
    case2_dict[itme][5] : item latent
    case2_dict[item][6] : 유저의 전체 행동데이터
    """
    drop_user_index = int(case2_dict[item][3][case2_dict[item][3]['idx'] == user_id].iloc[:,3]) # user_id에 해당하는 index 값 가져오기
    user_data = case2_dict[item][2].loc[drop_user_index] # 원본 행동 데이터에서 user_id에 해당하는 행을 가져옴
    user_history_non_indices = [int(i) for i in user_data[user_data <= 0].index.tolist()] # 유저가 평가하지 않은 아이템의 index를 가져옴
    user_predictions = case2_dict[item][0].loc[drop_user_index] # user_id에 해당하는 SGD 결과값을 가져온 후
    user_predictions_filtered = user_predictions.iloc[user_history_non_indices] # 유저가 평가하지 않은 아이템의 결과값만 뽑아옴
    sorted_predictions = user_predictions_filtered.sort_values(ascending=False) # SGD 결과값이 높은 순으로 정렬
    return sorted_predictions

def user_latent_cos(user_id, original_item, item_list, case2_dict):
    """
    original_item : 추천하려는 item
    """
    exist_action = False
    item = ''
    for item_category in item_list:
        if user_id in case2_dict[item_category][3]['idx'].values:
            exist_action = True
            item = item_category
            print(f'user {user_id}는 {item_category}에 대한 행동이 존재함')
            break
    if exist_action is False:
        #print(f'user {user_id}는 모든 아이템에 대한 행동이 없음')
        return -1
    else:
        print(f'{item} user latent에서 user {user_id}과 유사한 user 찾기')
        drop_user_place_index = int(case2_dict[item][3][case2_dict[item][3]['idx'] == user_id].iloc[:,3]) # user_id에 대한 index 번호 추출
        cosine_sim_matrix = cosine_similarity(case2_dict[item][4]) # user_latent에 대한 코사인 유사도 계산
        cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=case2_dict[item][4].index, columns=case2_dict[item][4].index)
        user_similarities = cosine_sim_df.loc[drop_user_place_index] # user_id의 코사인 유사도 값을 가져옴
        user_similarities[drop_user_place_index] = -1 # user_id를 선택하지 않도록 -1을 해줌
        sorted_user_similarities = user_similarities.sort_values(ascending=False) # 유사도가 높은 순으로 정렬
        # 유사도가 높은 유저를 순서대로 기존의 추천하려는 아이템에 행동이 있는지 확인
        for i in range(len(sorted_user_similarities)):
            most_similar_user_id = sorted_user_similarities.index[i]
            new_user_id = int(case2_dict[item][3][case2_dict[item][3].iloc[:,3] == most_similar_user_id].iloc[:,0]) # new_user_idx 찾기
            highest_similarity_score = sorted_user_similarities.iloc[i]
            if new_user_id in case2_dict[original_item][3]['idx'].values: # 기존에 추천하려는 아이템에 행동이 있다면 new_user_id로 선택됨
                new_user_id = new_user_id
                break
            else:
                print(f'{new_user_id}가 {original_item}에 대한 행동이 없음')
        print(f'user {user_id}과 가장 유사한 user : {new_user_id}, cos : {highest_similarity_score}')
        return new_user_id

def get_recommended_items(user_id, item, item_list, case2_dict, num_recom):
    if user_id in case2_dict[item][3]['idx'].values:
        recom_list = recommend_step(item, user_id, case2_dict)
        top_recommendations = recom_list.index.tolist()[:num_recom] # 상위 N개만큼 뽑아옴
        recommendations_result = case2_dict[item][1].iloc[top_recommendations]['idx'].tolist() # 아이템 idx 매핑
        return recommendations_result
    else:
        print(f'user {user_id}는 {item}에 대한 행동내역이 없음')
        item_list.remove(item) # 아이템 리스트에서 행동이 없는 아이템 제거
        new_user_id = user_latent_cos(user_id, item, item_list, case2_dict) # user_id와 가장 유사도가 높은 user_id를 탐색
        if new_user_id == -1:
            return -1
        else:   
            recom_list = recommend_step(item, new_user_id, case2_dict) # new_user_id에 대해 아이템 추천
            top_recommendations = recom_list.index.tolist()[:num_recom] # 상위 N개만큼 뽑아옴
            recommendations_result = case2_dict[item][1].iloc[top_recommendations]['idx'].tolist() # 아이템 idx 매핑
            return recommendations_result
    
# 모든 데이터 place, product, video 추천
def recommend_all(total_sgd_preds, user_id, total_df, ratings_df, idx, num_recom):
    if user_id in idx['idx'].values:
        print(f'{user_id}번 유저의 행동이 있습니다.')
        user_index = int(idx[idx['idx'] == user_id].iloc[:,2])

        # 원본 평점 데이터에서 user_id에 해당하는 행을 DataFrame으로 가져온다.
        user_data = ratings_df.loc[user_index]

        # 사용자가 이미 평가한 상품의 인덱스를 추출
        user_history_indices = [int(i) for i in user_data[user_data > 0].index.tolist()]
        user_history_non_indices = [int(i) for i in user_data[user_data <= 0].index.tolist()]
        #print(f'이미 평가한 아이템 길이: {len(user_history_indices)}')
        #print(len(user_history_non_indices),user_history_non_indices)
        non_recommendations = total_df.iloc[user_history_indices]['idx'].tolist()
        recommendations = total_df.iloc[user_history_non_indices]['idx'].tolist()
        #print("이미 평가한 아이템 길이, idx:", len(non_recommendations),non_recommendations)
        #print("평가 안한 아이템 길이, idx:", len(recommendations), recommendations)

        # SGD를 통해 예측된 사용자의 평점을 기반으로 데이터 정렬
        user_predictions = total_sgd_preds.loc[user_index]
        user_predictions_filtered = user_predictions.iloc[user_history_non_indices]
        sorted_predictions = user_predictions_filtered.sort_values(ascending=False)
        top_recommendations = sorted_predictions.index.tolist()[:num_recom]
        recommendations_result = total_df.iloc[top_recommendations]['idx'].tolist()
        print(f"user {user_id}에게 추천해줄 {10}개 아이템 idx : {recommendations_result}")
        return recommendations_result
    else:
        print(f'{user_id}번 유저의 행동이 없습니다.')

def item_latent_cos(user_id, item, case2_dict, num_recom):
    if user_id in case2_dict[item][3]['idx'].values:
        print(f'{item} item latent에서 user {user_id}이 행동했던 item과 유사한 item 찾기')
        drop_user_place_index = int(case2_dict[item][3][case2_dict[item][3]['idx'] == user_id].iloc[:,3]) # user_id에 대한 index 번호 추출
        all_item = case2_dict[item][2].loc[drop_user_place_index]
        non_zero_columns = all_item[all_item != 0].index.tolist()
        select_item = int(random.choice(non_zero_columns))
        cosine_sim_matrix = cosine_similarity(case2_dict[item][5]) # user_latent에 대한 코사인 유사도 계산
        cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=case2_dict[item][5].index, columns=case2_dict[item][5].index)
        user_similarities = cosine_sim_df.loc[select_item] # user_id의 코사인 유사도 값을 가져옴
        user_similarities[select_item] = -1 # user_id를 선택하지 않도록 -1을 해줌
        sorted_user_similarities = user_similarities.sort_values(ascending=False) # 유사도가 높은 순으로 정렬
        top_recommendations = sorted_user_similarities.index.tolist()[:num_recom] # 상위 N개만큼 뽑아옴
        recommendations_result = case2_dict[item][1].iloc[top_recommendations]['idx'].tolist() # 아이템 idx 매핑
        print(f'user {user_id}과 사용했던 아이템 {select_item} 와 유사도가 높은 idx: {recommendations_result}')
        return recommendations_result
    else:
        print(f'user {user_id}는 {item}에 대한 행동이 없습니다')

In [98]:
from interest_similarity import interest_similarity
#from get_recommended_items import get_recommended_items
#from load_data import *
# import warnings
# warnings.filterwarnings("ignore")

user_id = 4
new_user_item = [1,0,1]
new_user_outdoor = [0,1,0,0,0,0,0,0,0,1]
num_recom = 10
case2_dict = { 'place' : [case2_sgd_rating_place, place, case2_user_place, case2_user_place_idx, 
                          case2_user_latent_place, case2_item_latent_place, user_place],
         'video' : [case2_sgd_rating_video, video, case2_user_video, case2_user_video_idx, 
                    case2_user_latent_video, case2_item_latent_video, user_video],
         'product' : [case2_sgd_rating_product, product, case2_user_product, case2_user_product_idx, 
                      case2_user_latent_product, case2_item_latent_product, user_product]}
item = 'place'
item_list = list(case2_dict.keys())

# Case 2만 구현
# if : 기존 유저, else : 신규 유저 구분하기
if user_id in user_interest['idx'].values:
    recomm_list = get_recommended_items(user_id, item, item_list, case2_dict, num_recom)
    if recomm_list == -1:
        print(f'user {user_id}은 모든 item에 행동이 없습니다.')
    else:
        print(f"user {user_id}에게 추천해줄 {num_recom}개 {item} idx : {recomm_list}")
else:
    new_user_id = interest_similarity(item, case2_dict, user_interest, new_user_item, new_user_outdoor)
    if new_user_id == -1:
        print(f'user {user_id}은 모든 item에 행동이 없습니다.')
    else:
        recomm_list = get_recommended_items(new_user_id, item, item_list, case2_dict, num_recom)
        print(f"user {user_id}에게 추천해줄 {num_recom}개 {item} idx : {recomm_list}")

place에 행동이 존재하며 유사도가 가장 높은 인덱스는 25031이며, 유사도 점수는 0.7071067811865475입니다.
user 4에게 추천해줄 10개 place idx : [1376, 2382, 2592, 1442, 3630, 463, 693, 594, 2320, 705]
